In [1]:
import requests
import json

import os
import base64

In [2]:
def elastic_request(data=None, method=None, url=None):
    if method == None:
        method = requests.get
    if data:
        data = json.dumps(data)    

    return method(f"https://192.168.1.153:9200/{url}",
                  headers={
                    "Content-Type": "application/json",
                    "Accept": "application/json",
                    "Authorization": f"ApiKey {os.getenv('ELASTIC_API_KEY')}"
                  },
                  verify=False,
                  data=data)

In [ ]:
response = elastic_request(url="players-handbook-embedded/_search",
                           data={
                                "query": {
                                    "match": {
                                        "attachment.content": {
                                            "query": "Hunter's Mark",
                                            "boost": 0.7
                                        }
                                    }
                                },
                                "knn": {
                                    "field": "content-embedding",
                                    "k": 10,
                                    "boost": 0.3,
                                    "num_candidates": 10,
                                    "query_vector_builder": {
                                        "text_embedding": { 
                                            "model_id": "open-ai-embeddings", 
                                            "model_text": "Can I cast Hunter's Mark on an enemy who is invisible?" 
                                        }
                                    }
                                }
                           })

In [28]:
hits = response.json()["hits"]["hits"]

In [ ]:
[h["_source"]["attachment"]["content"] for h in hits]